In [1]:
import sys, os
from datetime import datetime, timedelta,date
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

In [2]:
products_df = pd.read_csv('./../data/amazon_baby.csv')

In [3]:
products_df.shape

(183531, 3)

In [4]:
products_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [5]:
products_df.review = products_df.review.fillna('')
products_df['reviews_clean'] = products_df.review.str.replace('[^\w\s]','').fillna('')
products_df = products_df[products_df.rating != 3]

In [6]:
products_df['sentiment'] = np.where(products_df['rating'] > 3, +1, -1)

In [7]:
products_df.head()

,name,review,rating,reviews_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [8]:
train_indexes = pd.read_json('../data/module-2-assignment-train-idx.json')
test_indexes = pd.read_json('../data/module-2-assignment-test-idx.json')

In [9]:
train_data = products_df.iloc[train_indexes[0].tolist()]
test_data = products_df.iloc[test_indexes[0].tolist()]

In [10]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

In [11]:
train_matrix = vectorizer.fit_transform(train_data['reviews_clean'])

In [12]:
test_matrix = vectorizer.transform(test_data['reviews_clean'])

In [13]:
sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
print("Number of Coefficients of this model are = {}".format(len(sentiment_model.coef_[0])))

Number of Coefficients of this model are = 121741


In [15]:
print("Number of Coefficients of this model are = {}".format(sum(sentiment_model.coef_[0] >= 0)))

Number of Coefficients of this model are = 85924


In [16]:
sample_test_data = test_data[10:13]
sample_test_data.head()

,name,review,rating,reviews_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [17]:
for i in range(3):
    print("Review {} is {}".format(i, sample_test_data.iloc[i]['review']))

Review 0 is Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.
Review 1 is Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.
Review 2 is Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it.


In [18]:
sample_test_matrix = vectorizer.transform(sample_test_data['reviews_clean'])

In [19]:
sentiment_model.decision_function(sample_test_matrix)

array([  5.60827718,  -3.15921163, -10.41343225])

In [20]:
def custom_predict(model, text_sparse_matrix):
    predictions = [1 if s >=0 else -1 for s in model.decision_function(text_sparse_matrix)]
    return np.array(predictions)

In [21]:
custom_predictions = pd.Series(custom_predict(sentiment_model, test_matrix))
sk_predictions = pd.Series(sentiment_model.predict(test_matrix))
sum(sk_predictions == custom_predictions)

33336

In [22]:
custom_predict(sentiment_model, sample_test_matrix)

array([ 1, -1, -1])

In [23]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

In [24]:
def prob(x):
    return 1/(1+ np.exp(-x))

In [25]:
def predict_probs(sentiment_model, test_matrix):
    return np.array([prob(s) for s in sentiment_model.decision_function(test_matrix)])

In [26]:
predict_probs(sentiment_model, sample_test_matrix)

array([9.96346018e-01, 4.07298447e-02, 3.00255379e-05])

In [27]:
class_predictions_test_data = sentiment_model.predict(test_matrix)

In [28]:
sentiment_model.predict_proba(test_matrix)

array([[2.20460442e-01, 7.79539558e-01],
       [7.44152354e-07, 9.99999256e-01],
       [6.62670976e-02, 9.33732902e-01],
       ...,
       [5.41656421e-06, 9.99994583e-01],
       [2.47500495e-06, 9.99997525e-01],
       [1.88335064e-02, 9.81166494e-01]])

In [29]:
test_data['positive_review_prediction'] = pd.Series([sentiment_model.predict_proba(row)[0][1] for row in test_matrix])

/Users/nehiljain/.pyenv/versions/3.4.6/envs/py34/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
test_data_pos = test_data.copy().sort_values(['positive_review_prediction'], ascending=False)

### Top 20 Positive Reviews 

In [79]:
'Safety 1st Exchangeable Tip 3 in 1 Thermometer' in test_data_pos.name.head(20).tolist()

False

In [31]:
test_data_pos[['name', 'positive_review_prediction']].head(20)

,name,positive_review_prediction
14482,Evenflo Whisper Connect Sensa Monitor,1.0
15732,"Fisher-Price Infant-To-Toddler Rocker, Blue/Green",1.0
18112,Bugaboo Frog Complete Stroller - Black,1.0
30535,Munchkin Five Sea Squirts,1.0
26830,"Leachco Wrap Strap Anywhere Safety Strap, Red",1.0
31271,"Prince Lionheart Wheely Bug, Ladybug, Large",1.0
18009,Prince Lionheart Jumbo Toy Hammock,1.0
20905,Boppy Newborn Lounger,1.0
12823,Baby Trend High Chair Palm Tree,1.0
32449,Aquatopia Deluxe Safety Bath Thermometer Alarm...,1.0


In [32]:
test_data_neg = test_data.copy().sort_values(['positive_review_prediction'], ascending=True)

### Top 20 Negative Reviews

In [33]:
test_data_neg[['name', 'positive_review_prediction']].head(20)

,name,positive_review_prediction
9655,Baby Einstein Seek &amp; Discover Activity Gym,3.454597e-11
31226,Sassy Teething Feeder and 16 Replacement Bags,6.165371e-10
7310,Avent Isis Manual Breast Pump,7.265702e-10
17222,Fisher-Price Ocean Wonders Aquarium Cradle Swing,3.101836e-09
17985,Prince Lionheart Jumbo Toy Hammock,7.318266e-09
13752,Medela Pump &amp; Save Breastmilk Bags - 50 pa...,2.634153e-08
13572,Mustela 2-In-1 Hair &amp; Body Shampoo 6.76 ou...,4.120121e-08
3747,Playtex Diaper Genie - First Refill Included,1.023396e-07
394,Baby Trend Diaper Champ,1.140153e-07
30538,"Munchkin 2 Pack Fresh Food Feeder, Colors May ...",1.223347e-07


### Evaluation

In [57]:
train_custom_preds = pd.Series(custom_predict(sentiment_model, train_matrix))
train_correct = 0
true_labels = train_data.sentiment.tolist()
for i in range(len(train_custom_preds)):
    train_correct += 1 if true_labels[i] == train_custom_preds[i] else 0
train_accuracy = round(train_correct/len(train_custom_preds), 2) 
print("Complex Model train data accuracy {}".format(train_accuracy))

Somplex Model train data accuracy 0.97


In [58]:
test_custom_preds = pd.Series(custom_predict(sentiment_model, test_matrix))
test_correct = 0
true_labels = test_data.sentiment.tolist()
for i in range(len(test_custom_preds)):
    test_correct += 1 if true_labels[i] == test_custom_preds[i] else 0
test_accuracy = round(test_correct/len(test_custom_preds), 2) 
print("Complex Model test data accuracy {}".format(train_accuracy))

Complex Model test data accuracy 0.97


### Simpler Model

In [35]:
significant_words_vocab = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [38]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words_vocab) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['reviews_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['reviews_clean'])

In [39]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
print("Number of Coefficients of the Simple model are = {}".format(len(simple_model.coef_[0])))

Number of Coefficients of the Simple model are = 20


In [41]:
print("Number of Coefficients of the Simple model are = {}".format(sum(simple_model.coef_[0] >= 0)))

Number of Coefficients of the Simple model are = 10


In [47]:
simple_model_coef_table = (pd.DataFrame({'word': significant_words_vocab, 'coefficient': simple_model.coef_[0]})
                           .sort_values(['coefficient'], ascending=False)
                          )

In [50]:
simple_model_coef_table[simple_model_coef_table.coefficient >= 0]

,coefficient,word
6,1.673074,loves
5,1.509812,perfect
0,1.363690,love
2,1.192538,easy
1,0.944000,great
4,0.520186,little
7,0.503760,well
8,0.190909,able
3,0.085513,old
9,0.058855,car


In [63]:
simple_custom_predictions = pd.Series(custom_predict(simple_model, test_matrix_word_subset))
test_simple_correct = 0
true_labels = test_data.sentiment.tolist()
for i in range(len(simple_custom_predictions)):
    test_simple_correct += 1 if true_labels[i] == simple_custom_predictions[i] else 0
test_simple_accuracy = round(test_simple_correct/len(simple_custom_predictions), 2) 
print("Simple Model Test data accuracy {}".format(test_simple_accuracy))

Simple Model Test data accuracy 0.87


In [65]:
simple_custom_predictions = pd.Series(custom_predict(simple_model, train_matrix_word_subset))
train_simple_correct = 0
true_labels = train_data.sentiment.tolist()
for i in range(len(simple_custom_predictions)):
    train_simple_correct += 1 if true_labels[i] == simple_custom_predictions[i] else 0
train_simple_accuracy = round(train_simple_correct/len(simple_custom_predictions), 2) 
print("Simple Model Training data accuracy {}".format(train_simple_accuracy))

Simple Model Training data accuracy 0.87


### Basline Model

In [66]:
total_positives = sum(test_data.sentiment == 1)

In [71]:
baseline_accuracy = round(total_positives/len(test_data.sentiment.tolist()),2)

In [73]:
print("Baseline Majority model accuracy {}".format(baseline_accuracy))

Baseline Majority model accuracy 0.84
